https://stepik.org/lesson/303592/step/9

In [ ]:
import torch
import torch.nn as nn

channel_count = 6
eps = 1e-3
batch_size = 20
input_size = 2

input_tensor = torch.randn(batch_size, channel_count, input_size)


def custom_group_norm(input_tensor, groups, eps):
    # Напишите в этом месте нормирование тензора

    custom_group_normed_tensor = torch.zeros_like(input_tensor)
    step = channel_count // groups
    for batch in range(0, batch_size):
        for i in range(0, channel_count, step):

            # Способ 1 (не работает на платформе Stepik)
            # mean = input_tensor[(batch,), i:i + step, :].mean(dim=[1, 2])
            # var = input_tensor[(batch,), i:i + step, :].var(dim=[1, 2], unbiased=False)

            # Способ 2:
            mean = input_tensor[(batch,), i:i + step, :].reshape(-1).mean(dim=0)
            var = input_tensor[(batch,), i:i + step, :].reshape(-1).var(dim=0, unbiased=False)

            normed_region = (input_tensor[batch, i:i + step, :] - mean) / torch.sqrt(var + eps)
            custom_group_normed_tensor[batch, i:i + step, :] = normed_region

    return custom_group_normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True
for groups in [1, 2, 3, 6]:
    group_norm = nn.GroupNorm(groups, channel_count, eps=eps, affine=False)
    norm_output = group_norm(input_tensor)
    custom_output = custom_group_norm(input_tensor, groups, eps)
    all_correct &= torch.allclose(norm_output, custom_output, 1e-3)
    all_correct &= norm_output.shape == custom_output.shape
    # if groups == 1:
    #     print(norm_output[0][1])
    #     print(custom_output[0][1])
print(all_correct)

True
